# <font color = 'green'>Purpose of this Notebook: to clean the raw GCSE data and create a dataset for use in modeling</font>

## Index

<a href='#nb_setup'>Notebook Set Up</a><br />
1)  <a href='#1'>Create a DataFrame with 2017-18 (all base) data</a><br />
2)  <a href='#2'>Remove Special Schools and rows of summary data</a><br />
3)  <a href='#3'>Remove invalid values for target y (Field: P8MEA)</a><br />
4)  <a href='#4'>Examine number of Nulls / NaNs per column and row, and format the target variable correctly</a><br />
5)  <a href='#5'>Create a table to show which data in which columns need to be cleaned</a><br />
6)  <a href='#6'>Decide how to handle remaining non-numeric data</a><br />
7)  <a href='#7'>Action the decisions made in 6) above, to make data modelable</a><br />
8)  <a href='#8'>Convert all relevant columns to floats</a><br />
9)  <a href='#9'>Export final (base) dataset</a><br />

<br />


<a id='nb_setup'></a>

# <font color='blue'>Notebook Set Up (import modules and data)</font>

In [1]:
# import the libraries and modules that will be needed

import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# added additional modules that will be needed
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from pandas import read_csv
import csv

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
# import base data

df = read_csv('data_files/england_ks4final17_18.csv')
df_ks4_meta = read_csv('data_files/_ks4final_meta.csv')


/Users/jacktyler-whittle/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5673 entries, 0 to 5672
Columns: 496 entries, RECTYPE to P8_BANDING
dtypes: float64(12), int64(1), object(483)
memory usage: 21.5+ MB


In [4]:
# Summary of school types (1=mainstream school; 2=special school; 4=local authority; 5=National (all schools); 7=National (maintained schools))

df['RECTYPE'].value_counts()

1    4328
2    1191
4     152
7       1
5       1
Name: RECTYPE, dtype: int64

<a id='1'></a>

# <font color='blue'>1) Create a DataFrame with 2017-18 (all base) data</font>
Dataset goes from 496 to 279 columns

- Set up DataFrame for 2017-18 data (name: df2)
- **Decisions**<br />
   1) To avoid autocorrelation, exclude data that is duplicating data represented already, but in a slightly different manner.  e.g.:
     - Exclude columns containing numbers which are represented as percentages in another column (e.g. exclude 
    'Number of boys at the end of key stage 4' and include only 'Percentage of pupils at the end of key stage 4 who are boys')<br />
     - Exclude data showing percentage of girls, and include only data showing percentage of boys, as one can be directly inferred from the other<br />

  2) Exclude columns containing upper and lower confidence intervals for measures<br />
   3) Exclude data on previous years, as this model was to explain variation within the year's results (time series analysis will be covered in a future phase of this project).<br />

**Note**: all columns, with their descriptions, are noted in ['GCSEs_Key_Meta_Sep-19.xls']('GCSEs_Key_Meta_Sep-19.xls').  The columns highlighted in green in 'Include in Base Model' (column J) are those which were included in the base model below.

In [5]:
# Create DataFrame

df2 = df[[
'RECTYPE',
'LEA',
'ESTAB',
'URN',
'SCHNAME',
'SCHNAME_AC',
'PCODE',
'ICLOSE',
'NFTYPE',
'RELDENOM',
'ADMPOL',
'ADMPOL_2017',
'EGENDER',
'FEEDER',
'TABKS2',
'TAB1618',
'TOTPUPS',
'NUMBOYS',
'NUMGIRLS',
'TPUP',
'PBPUP',
'KS2APS',
'PTPRIORLO',
'PTPRIORAV',
'PTPRIORHI',
'PTFSM6CLA1A',
'PTNOTFSM6CLA1A',
'PTEALGRP2',
'PTEALGRP1',
'PTEALGRP3',
'PTNMOB',
'PSENSE4',
'PSENAPK4',
'ATT8SCR',
'ATT8SCRENG',
'ATT8SCRMAT',
'ATT8SCREBAC',
'ATT8SCROPEN',
'ATT8SCROPENG',
'ATT8SCROPENNG',
'AVGEBACFILL',
'AVGOPENFILL',
'P8PUP',
'TP8ADJ',
'P8MEACOV',
'P8MEA',
'P8CILOW',
'P8CIUPP',
'P8MEA_ORIG',
'P8MEAENG',
'P8MEAMAT',
'P8MEAEBAC',
'P8MEAOPEN',
'PTL2BASICS_94',
'PTL2BASICS_95',
'TOTEBACCAPS',
'EBACCAPS',
'EBACCAPS_FSM6CLA1A',
'EBACCAPS_NFSM6CLA1A',
'EBACCAPS_LO',
'EBACCAPS_AV',
'EBACCAPS_HI',
'EBACCAPS_EAL',
'EBACCAPS_GIRLS',
'EBACCAPS_NMOB',
'PTEBACC_E_PTQ_EE',
'PTEBACC_94',
'PTEBACC_95',
'PTEBACENG_E_PTQ_EE',
'PTEBACMAT_E_PTQ_EE',
'PTEBAC2SCI_E_PTQ_EE',
'PTEBACHUM_E_PTQ_EE',
'PTEBACLAN_E_PTQ_EE',
'PTEBACENG_94',
'PTEBACENG_95',
'PTEBACMAT_94',
'PTEBACMAT_95',
'PTEBAC2SCI_94',
'PTEBAC2SCI_95',
'PTEBACHUM_94',
'PTEBACHUM_95',
'PTEBACLAN_94',
'PTEBACLAN_95',
'SCIVACOV_PTQ_EE',
'HUMVACOV_PTQ_EE',
'LANVACOV_PTQ_EE',
'SCIVAMEA_PTQ_EE',
'HUMVAMEA_PTQ_EE',
'LANVAMEA_PTQ_EE',
'TEBACENG_94',
'TEBACENG_95',
'TEBACMAT_94',
'TEBACMAT_95',
'TEBAC2SCI_94',
'TEBAC2SCI_95',
'TEBACHUM_94',
'TEBACHUM_95',
'TEBACLAN_94',
'TEBACLAN_95',
'PTEBACC91',
'PTEBACENG91',
'PTEBACMAT91',
'PTEBAC2SCI91',
'PTEBACHUM91',
'PTEBACLAN91',
'ATT8SCR_FSM6CLA1A',
'P8PUP_FSM6CLA1A',
'TP8ADJ_FSM6CLA1A',
'P8MEA_FSM6CLA1A',
'P8MEA_FSM6CLA1A_ORIG',
'ATT8SCR_NFSM6CLA1A',
'P8PUP_NFSM6CLA1A',
'TP8ADJ_NFSM6CLA1A',
'P8MEA_NFSM6CLA1A',
'P8MEA_NFSM6CLA1A_ORIG',
'ATT8SCRENG_FSM6CLA1A',
'P8MEAENG_FSM6CLA1A',
'ATT8SCRMAT_FSM6CLA1A',
'P8MEAMAT_FSM6CLA1A',
'ATT8SCREBAC_FSM6CLA1A',
'P8MEAEBAC_FSM6CLA1A',
'ATT8SCROPEN_FSM6CLA1A',
'P8MEAOPEN_FSM6CLA1A',
'ATT8SCRENG_NFSM6CLA1A',
'P8MEAENG_NFSM6CLA1A',
'ATT8SCRMAT_NFSM6CLA1A',
'P8MEAMAT_NFSM6CLA1A',
'ATT8SCREBAC_NFSM6CLA1A',
'P8MEAEBAC_NFSM6CLA1A',
'ATT8SCROPEN_NFSM6CLA1A',
'P8MEAOPEN_NFSM6CLA1A',
'ATT8SCROPENG_FSM6CLA1A',
'ATT8SCROPENNG_FSM6CLA1A',
'ATT8SCROPENG_NFSM6CLA1A',
'ATT8SCROPENNG_NFSM6CLA1A',
'DIFFN_ATT8',
'DIFFN_P8MEA',
'ATT8SCR_LO',
'P8PUP_LO',
'TP8ADJ_LO',
'P8MEA_LO',
'P8MEA_LO_ORIG',
'ATT8SCR_AV',
'P8PUP_AV',
'TP8ADJ_AV',
'P8MEA_AV',
'P8MEA_AV_ORIG',
'ATT8SCR_HI',
'P8PUP_HI',
'TP8ADJ_HI',
'P8MEA_HI',
'P8MEA_HI_ORIG',
'ATT8SCR_EAL',
'ATT8SCRENG_EAL',
'ATT8SCRMAT_EAL',
'ATT8SCREBAC_EAL',
'ATT8SCROPEN_EAL',
'ATT8SCROPENG_EAL',
'ATT8SCROPENNG_EAL',
'P8PUP_EAL',
'TP8ADJ_EAL',
'P8MEA_EAL',
'P8MEA_EAL_ORIG',
'ATT8SCR_GIRLS',
'ATT8SCRENG_GIRLS',
'ATT8SCRMAT_GIRLS',
'ATT8SCREBAC_GIRLS',
'ATT8SCROPEN_GIRLS',
'ATT8SCROPENG_GIRLS',
'ATT8SCROPENNG_GIRLS',
'P8PUP_GIRLS',
'TP8ADJ_GIRLS',
'P8MEA_GIRLS',
'P8MEA_GIRLS_ORIG',
'ATT8SCR_BOYS',
'ATT8SCRENG_BOYS',
'ATT8SCRMAT_BOYS',
'ATT8SCREBAC_BOYS',
'ATT8SCROPEN_BOYS',
'ATT8SCROPENG_BOYS',
'ATT8SCROPENNG_BOYS',
'P8PUP_BOYS',
'TP8ADJ_BOYS',
'P8MEA_BOYS',
'P8MEA_BOYS_ORIG',
'ATT8SCR_NMOB',
'ATT8SCRENG_NMOB',
'ATT8SCRMAT_NMOB',
'ATT8SCREBAC_NMOB',
'ATT8SCROPEN_NMOB',
'ATT8SCROPENG_NMOB',
'ATT8SCROPENNG_NMOB',
'P8PUP_NMOB',
'TP8ADJ_NMOB',
'P8MEA_NMOB',
'P8MEA_NMOB_ORIG',
'TEBACC_ELO_PTQ_EE',
'PTEBACC_ELO_PTQ_EE',
'PTEBACCLO_94',
'PTEBACCLO_95',
'TEBACC_EAV_PTQ_EE',
'PTEBACC_EAV_PTQ_EE',
'PTEBACCAV_94',
'PTEBACCAV_95',
'TEBACC_EHI_PTQ_EE',
'PTEBACC_EHI_PTQ_EE',
'PTEBACCHI_94',
'PTEBACCHI_95',
'PTEBACC_EFSM6CLA1A_PTQ_EE',
'PTEBACC_ENFSM6CLA1A_PTQ_EE',
'PTEBACC_94_FSM6CLA1A',
'PTEBACC_95_FSM6CLA1A',
'PTEBACC_94_NFSM6CLA1A',
'PTEBACC_95_NFSM6CLA1A',
'SCIVAMEA_LO_PTQ_EE',
'SCIVAMEA_AV_PTQ_EE',
'SCIVAMEA_HI_PTQ_EE',
'SCIVAMEA_FSM6CLA1A_PTQ_EE',
'SCIVAMEA_NFSM6CLA1A_PTQ_EE',
'HUMVAMEA_LO_PTQ_EE',
'HUMVAMEA_AV_PTQ_EE',
'HUMVAMEA_HI_PTQ_EE',
'HUMVAMEA_FSM6CLA1A_PTQ_EE',
'HUMVAMEA_NFSM6CLA1A_PTQ_EE',
'LANVAMEA_LO_PTQ_EE',
'LANVAMEA_AV_PTQ_EE',
'LANVAMEA_HI_PTQ_EE',
'LANVAMEA_FSM6CLA1A_PTQ_EE',
'LANVAMEA_NFSM6CLA1A_PTQ_EE',
'PTEBACC_94_17',
'PTEBACC_95_17',
'PBEBACC_E_PTQ_EE',
'PBEBACC_94',
'PBEBACC_95',
'PGEBACC_E_PTQ_EE',
'PGEBACC_94',
'PGEBACC_95',
'PTEBACC_ENMOB_PTQ_EE',
'PTEBACCNMOB_94',
'PTEBACCNMOB_95',
'PTEBACC_EEAL_PTQ_EE',
'PTEBACCEAL_94',
'PTEBACCEAL_95',
'PT5EM_94',
'PT5EM_94_18',
'PTANYQ_PTQ_EE',
'PTFSM6CLA1ABASICS_94',
'PTNOTFSM6CLA1ABASICS_94',
'PTBASICSLO_94',
'PTBASICSAV_94',
'PTBASICSHI_94',
'PBL2BASICS_94',
'PGL2BASICS_94',
'PTL2BASICSEAL_94',
'PTL2BASICSNMOB_94',
'PTFSM6CLA1ABASICS_95',
'PTNOTFSM6CLA1ABASICS_95',
'PTBASICSLO_95',
'PTBASICSAV_95',
'PTBASICSHI_95',
'PBL2BASICS_95',
'PGL2BASICS_95',
'PTL2BASICSEAL_95',
'PTL2BASICSNMOB_95',
'PTmultiLan_E',
'PTtripleSci_E',
'TAVENT_E_3NG_PTQ_EE',
'TAVENT_E_3NG_LO_PTQ_EE',
'TAVENT_E_3NG_AV_PTQ_EE',
'TAVENT_E_3NG_HI_PTQ_EE',
'TAVENT_E_3NG_FSM6CLA1A_PTQ_EE',
'TAVENT_E_3NG_NFSM6CLA1A_PTQ_EE',
'TAVENT_G_PTQ_EE',
'TAVENT_GLO_PTQ_EE',
'TAVENT_GAV_PTQ_EE',
'TAVENT_GHI_PTQ_EE',
'TAVENT_GFSM6CLA1A_PTQ_EE',
'TAVENT_GNFSM6CLA1A_PTQ_EE',
'P8_BANDING',
]]


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5673 entries, 0 to 5672
Columns: 279 entries, RECTYPE to P8_BANDING
dtypes: float64(8), int64(1), object(270)
memory usage: 12.1+ MB


In [7]:
df2.head(3)

,RECTYPE,LEA,ESTAB,URN,SCHNAME,SCHNAME_AC,PCODE,ICLOSE,NFTYPE,RELDENOM,...,TAVENT_E_3NG_HI_PTQ_EE,TAVENT_E_3NG_FSM6CLA1A_PTQ_EE,TAVENT_E_3NG_NFSM6CLA1A_PTQ_EE,TAVENT_G_PTQ_EE,TAVENT_GLO_PTQ_EE,TAVENT_GAV_PTQ_EE,TAVENT_GHI_PTQ_EE,TAVENT_GFSM6CLA1A_PTQ_EE,TAVENT_GNFSM6CLA1A_PTQ_EE,P8_BANDING
0,1,202.0,4104.0,100049.0,Haverstock School,NaN,NW3 2BQ,0.0,CY,Does not apply,...,7.7,7.2,7.4,7.1,6.3,6.9,7.7,7,7.3,3
1,1,202.0,4166.0,100050.0,Parliament Hill School,NaN,NW5 1RL,0.0,CY,Does not apply,...,8.8,8.6,8.4,8.4,6.9,8.1,8.7,8.5,8.3,1
2,1,210.0,4005.0,145313.0,Bacon's College,NaN,SE16 6AT,0.0,AC,Church of England,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## <font color='green'>Summary stats for all England</font>

In [8]:
# National Data - all schools (5430) and maintained schools (5431)

df2.loc[5430:5431, ('P8MEA', 'P8MEA_ORIG', 'ATT8SCR','TAVENT_E_3NG_PTQ_EE','TAVENT_G_PTQ_EE','TOTPUPS','TPUP', 'PT5EM_94')]

,P8MEA,P8MEA_ORIG,ATT8SCR,TAVENT_E_3NG_PTQ_EE,TAVENT_G_PTQ_EE,TOTPUPS,TPUP,PT5EM_94
5430,NaN,NaN,44.5,7.8,7.4,3709343,583615.0,55.50%
5431,-0.02,-0.03,46.5,8.3,7.8,3290097,523636.0,60.10%


In [9]:
# Average attainment points per average number of GCSEs taken:
    
# All schools:
print("All Schools: "+ str(44.5/7.4))

# All maintained schools:
print("All Maintained Schools: "+ str(46.5/7.8))

All Schools: 6.013513513513513
All Maintained Schools: 5.961538461538462


<a id='2'></a>

# <font color='blue'>2) Remove Special Schools and rows of summary data</blue>
Dataset goes from 5,673 entries (rows) to 4,328 schools to model (rows)

**Decision**: remove Special Schools as they are by definition different from other secondary schools; also remove all rows of data that are summary data (and not data for specific schools)

In [10]:
# Remove the special needs schools (RECTYPE 1)

df2['RECTYPE'].value_counts()

1    4328
2    1191
4     152
7       1
5       1
Name: RECTYPE, dtype: int64

In [11]:
df3 = df2[df2['RECTYPE'] == 1]

In [12]:
df3['RECTYPE'].value_counts()

1    4328
Name: RECTYPE, dtype: int64

<a id='3'></a>

# <font color='blue'>3) Remove invalid values for target y (Field: P8MEA)</blue>

Goes from 4,328 to 3,165 rows (schools)

In [13]:
# Check that the Progress 8 Banding is not provided for schools which are not reporting a Progress 8 score

df3[df3['P8MEA'].isin(['NP', 'NE', 'SUPP', 'LOWCOV', 'NEW'])]['P8_BANDING'].value_counts()

SUPP    4
Name: P8_BANDING, dtype: int64

In [14]:
# remove invalid values for target y (P8MEA)

df4 = df3[~df3['P8MEA'].isin(['NP', 'NE', 'SUPP', 'LOWCOV', 'NEW'])]
len(df4)

3393

In [15]:
# drop NaNs in P8MEA column (as there needs to be a target y for each school that is part of the model)
df4 = df4.dropna(subset=['P8MEA'])

In [16]:
# see number of schools now in dataset
len(df4)

3165

<a id='4'></a>

# <font color='blue'>4) Examine number of Nulls / NaNs per column and row, and format the target variable correctly</blue>
Dataset stays at 3,165 rows (schools)

In [17]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3165 entries, 0 to 5580
Columns: 279 entries, RECTYPE to P8_BANDING
dtypes: float64(8), int64(1), object(270)
memory usage: 6.8+ MB


### <font color='green'>a) Number of NaNs per column and per row</font>

In [18]:
# Look at number of Nulls / NaNs per column

df_column_nulls = pd.DataFrame(df4.isnull().sum().sort_values(ascending=False), columns=['number_nulls'])
df_column_nulls.head()

,number_nulls
SCHNAME_AC,3073
PBPUP,200
ATT8SCR_BOYS,200
P8PUP_BOYS,200
P8MEA_BOYS,200


In [19]:
# Look at number of Nulls / NaNs per row

df4.isnull().sum(axis=1).sort_values(ascending=False).head(10)

3305    7
1212    6
3424    6
3434    6
3433    6
3431    6
3429    6
3986    6
3423    6
165     6
dtype: int64

###  <font color='green'>b) Remove columns and rows with a very high volume of NaNs</font>

In [20]:
# remove Revised School Name (for Schools converted to Academy since Sep 2015) as it is not relevant for
# analysis of 2018 data; all other rows and columns have small enough volume of NaNs
# to see how they can be imputed

df4.drop('SCHNAME_AC', axis=1, inplace=True)

###  <font color='green'>c) Format the target y column correctly</font>

In [21]:
# format the Target y column correctly

df4['P8MEA'] = df4['P8MEA'].astype(float)

<a id='5'></a>

# <font color='blue'>5) Create a table to show which data in which columns need to be cleaned</font>

In [22]:
df5 = df4.copy()

### <font color='green'>a) Identify which columns (that should be floats) have the following keywords in their columns (these are words highlighted in the meta data provided as ones which are used in the file):</font>

- NEW
- NE
- NA
- SUPP
- NP
- LOWCOV


In [23]:
# write a function to create a DataFrame containing a list of all the columns in the base dataset, and how many
# of each text type abbreviations are being used in place of values

def item_freq_in_col(df, list_words, sort_by='total_freq'):
    '''Input: dataframe and the 'items' you want to count (in a list)
    Returns new Dataframe with:
    1) Columns from original dataframe as rows
    2) Number of NaNs per column ('# nulls')
    3) Frequency of 'items' from each column listed in a separate column per 'item'
    4) Total (sum) of all text items for that column
    5) % of rows that the text values represent
    '''
    
    df_item_freq = pd.DataFrame({'column_name':df.columns})
    
    df_item_freq['# nulls'] = [df[col].isnull().sum() for col in df.columns]
    
    for word in list_words:
        dict = {i: np.sum(df[i] == word) for i in df.columns}
        df_item_freq['# "' + word + '"'] = dict.values()
        
    df_item_freq['total_freq'] = df_item_freq.sum(axis=1)
    df_item_freq['pct_all_rows'] = df_item_freq['total_freq'].apply(lambda x: round(x/len(df)*100))
    df_item_freq.sort_values(sort_by, ascending=False, inplace=True)  
    return df_item_freq


In [24]:
df_item_freq = item_freq_in_col(df5,['NE','SUPP', 'NEW', 'NA', 'LOWCOV', 'NP'])
df_item_freq.head(3)

/Users/jacktyler-whittle/anaconda3/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
223,LANVAMEA_LO_PTQ_EE,0,752,1830,0,0,5,0,2587,82
160,P8MEA_EAL,0,150,1168,0,0,0,15,1333,42
159,TP8ADJ_EAL,0,165,1168,0,0,0,0,1333,42


### <font color='green'>b) Add in a description of each column to help determine what to do with it</font>

In [25]:
# 1) review csv of column descriptions

df_ks4_meta.head(3)

,Column,Metafile heading,Metafile description,Methodology changes,Null field for special schools,Null field for local authority records,Null field for National (all schools) records,Null field for National (maintained schools) records
0,1,RECTYPE,Record type (1=mainstream school; 2=special sc...,NaN,NaN,NaN,NaN,NaN
1,2,ALPHAIND,Alphabetic sorting index,NaN,NaN,Yes,Yes,Yes
2,3,LEA,Local authority code (see separate list of loc...,NaN,NaN,NaN,Yes,Yes


In [26]:
# 2) rename columns to align with naming convertion used in 'df_item_freq' DataFrame above

df_ks4_meta_2 = df_ks4_meta[['Metafile heading', 'Metafile description']]
df_ks4_meta_2.columns = ['column_name', 'column_description']
df_ks4_meta_2.head(3)

,column_name,column_description
0,RECTYPE,Record type (1=mainstream school; 2=special sc...
1,ALPHAIND,Alphabetic sorting index
2,LEA,Local authority code (see separate list of loc...


In [27]:
# 3) merge df of column descriptions with the 'df_item_freq' DataFrame (note: left_index keeps the index)

df_data_issues = pd.merge(df_item_freq, df_ks4_meta_2, on='column_name', how='left', left_index=True)
df_data_issues.head(3)


,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows,column_description
365,LANVAMEA_LO_PTQ_EE,0,752,1830,0,0,5,0,2587,82,English Baccalaureate Languages Value Added me...
256,P8MEA_EAL,0,150,1168,0,0,0,15,1333,42,Adjusted Progress 8 measure - pupils for whom ...
255,TP8ADJ_EAL,0,165,1168,0,0,0,0,1333,42,Number of pupils for whom English is an additi...


In [28]:
# check that there are 'Total Pupils' for all schools in the dataset

df_data_issues[df_data_issues['column_name'] == 'TOTPUPS']

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows,column_description
25,TOTPUPS,0,0,0,0,0,0,15,15,0,Number of pupils on roll (all ages)


In [29]:
# number of columns containing non-numeric (unmodelable) data 

df_data_issues[df_data_issues['# "SUPP"'] > 0].count()

column_name           196
# nulls               196
# "NE"                196
# "SUPP"              196
# "NEW"               196
# "NA"                196
# "LOWCOV"            196
# "NP"                196
total_freq            196
pct_all_rows          196
column_description    196
dtype: int64

In [30]:
len(df5)

3165

<a id='6'></a>

# <font color='blue'>6) Decide how to handle remaining non-numeric data</font>
To maximise number of schools and features being modeled, without compromising on quality of data

### <font color='green'>a) Determine whether Boys and Girls' NaN values should be replaced with zeroes, as they represent single sex schools</font>

In [31]:
# of all the schools with Number of Boys as NaN, how many are Girls only schools?
# A: all of them

filtered_df = df5[df5['NUMBOYS'].isnull()]
filtered_df['EGENDER'].value_counts()

GIRLS    115
Name: EGENDER, dtype: int64

In [32]:
# of all the schools with Number of Girls as NaN, how many are Boys only schools?
# A: all of them

filtered_df = df5[df5['NUMGIRLS'].isnull()]
filtered_df['EGENDER'].value_counts()

BOYS    148
Name: EGENDER, dtype: int64

**DECISION**: replace NaN values with 0 for the above, and other related fields (see section 7 below)

### <font color='green'>b ) Determine what to do with the 'Admissions Policy' columns</font>

In [33]:
df5['ADMPOL'].value_counts()

COMP    2864
SEL      162
MOD      110
UK         2
Name: ADMPOL, dtype: int64

In [34]:
df5['ADMPOL_2017'].value_counts()

OTHER NON SEL                 2759
NON SEL IN HIGHLY SEL AREA     215
SEL                            162
Name: ADMPOL_2017, dtype: int64

**DECISION**: change blanks to 'OTHER NON SEL' (since 'SEL' - Selective Schools - is fully accounted for in the data (all Grammar Schools in the UK)) (see section 7 below)

### <font color='green'>c) Determine how to handle values for 'Suppressed' Progress 8 Banding values</font>

In [35]:
# Look at the distribution of Progress 8 Banding

df5['P8_BANDING'].value_counts()

# don't need to remove the 'SUPP' rows as they have a P8MEA, so Banding can be inferred

3       1126
4        594
2        512
1        411
5        377
SUPP     145
Name: P8_BANDING, dtype: int64

In [36]:
# Look at schools with Progress 8 Banding 'Suppressed' (Value: 'SUPP'), which indicates that
# a school or college's figures have been suppressed because there are 5 or fewer pupils in the cohort

df5a = df5[df5['P8_BANDING'] == 'SUPP']

df5b = df5a.loc[:, ('URN','P8MEA', 'P8_BANDING', 'LEA', 'TPUP', 'P8CILOW', 'P8CIUPP')].sort_values('TPUP')
df5b.head()

,URN,P8MEA,P8_BANDING,LEA,TPUP,P8CILOW,P8CIUPP
3421,139895.0,-2.99,SUPP,884.0,10.0,-3.77,-2.21
1206,141012.0,-0.80,SUPP,855.0,10.0,-1.58,-0.01
1172,140941.0,-1.48,SUPP,312.0,14.0,-2.19,-0.76
3768,141035.0,-0.59,SUPP,312.0,16.0,-1.25,0.07
3388,139590.0,-0.87,SUPP,861.0,17.0,-1.47,-0.27


<img src="pics/banding.png" alt="Drawing" style="width: 400px;" align="left"/>

**DECISION**: Allocate each 'SUPP' school as either 1, 3 or 5 Banding, based on a simplified version of the government assesment for scoring:
-   P8 Banding = 1: if P8MEA score >0.5
-   P8 Banding = 3: if -0.5 < P8MEA score < 0.5
-   P8 Banding = 5: if P8MEA score < -0.5

### <font color='red'>Decisions:</font>

Previously made Decision: a row must have a School_ID (column name: URN) and Progress 8 score (column name: P8MEA) to be included (otherwise it cannot be identified in the data)<br />

i) Replace NaNs for numbers of Boys and Girls with zeroes.<br />
ii) Replace girls-related fields that have NP with 0 because they are girls-only (checked with % girls / boys column).  Similarly for boys.<br />
iii) Admissions Policy: change blanks to 'OTHER NON SEL'<br />
iv) P8 Banding: allocate each 'SUPP' school as either 1, 3 or 5 Banding, based on government scoring<br />
v)  Remove any remaining column that has non-numerics (NaNs) in them (excluding non-numeric columns) since the data cannot be inferred.<br />

<a id='7'></a>

# <font color='blue'>7) Action the decisions made in 6) above, to make data modelable</font>

In [37]:
df6 = df5.copy()

### <font color='green'>i) Replace NaNs for numbers of Boys and Girls with zeroes</font>

In [38]:
df6['PBPUP'] = df6['PBPUP'].fillna('0%')
df6['P8PUP_BOYS'] = df6['P8PUP_BOYS'].fillna('0%')
df6['ATT8SCR_BOYS'] = df6['ATT8SCR_BOYS'].fillna('0%')
df6['P8MEA_BOYS'] = df6['P8MEA_BOYS'].fillna('0%')

df6['ATT8SCR_GIRLS'] = df6['ATT8SCR_GIRLS'].fillna('0%')
df6['P8MEA_GIRLS'] = df6['P8MEA_GIRLS'].fillna('0%')
df6['P8PUP_GIRLS'] = df6['P8PUP_GIRLS'].fillna('0%')
df6['NUMGIRLS'] = df6['NUMGIRLS'].fillna('0%')
df6['EBACCAPS_GIRLS'] = df6['EBACCAPS_GIRLS'].fillna('0%')

df6['NUMBOYS'] = df6['NUMBOYS'].fillna('0%')

### <font color='green'>ii) Replace girls-related fields that have NP with 0 because they are girls-only (checked with % girls / boys column).  Similarly for boys.</font>

In [39]:
df6[(df6['ATT8SCREBAC_BOYS'] == 'NE') & (df6['EGENDER'] == 'GIRLS')]

,RECTYPE,LEA,ESTAB,URN,SCHNAME,PCODE,ICLOSE,NFTYPE,RELDENOM,ADMPOL,...,TAVENT_E_3NG_HI_PTQ_EE,TAVENT_E_3NG_FSM6CLA1A_PTQ_EE,TAVENT_E_3NG_NFSM6CLA1A_PTQ_EE,TAVENT_G_PTQ_EE,TAVENT_GLO_PTQ_EE,TAVENT_GAV_PTQ_EE,TAVENT_GHI_PTQ_EE,TAVENT_GFSM6CLA1A_PTQ_EE,TAVENT_GNFSM6CLA1A_PTQ_EE,P8_BANDING
1,1,202.0,4166.0,100050.0,Parliament Hill School,NW5 1RL,0.0,CY,Does not apply,COMP,...,8.8,8.6,8.4,8.4,6.9,8.1,8.7,8.5,8.3,1
7,1,202.0,5401.0,100059.0,La Sainte Union Catholic Secondary School,NW5 1RP,0.0,VA,Roman Catholic,COMP,...,8.6,8.3,8.5,8.4,7.7,8.2,8.6,8.3,8.5,1
13,1,203.0,4682.0,100193.0,St Ursula's Convent School,SE10 8HN,0.0,VA,Roman Catholic,COMP,...,10.7,9.7,10,9.9,8,9,10.7,9.7,10,1
16,1,204.0,4302.0,137442.0,Clapton Girls' Academy,E5 0RB,0.0,ACC,Does not apply,COMP,...,9.2,8.1,8.9,8.2,5.2,7.7,9.1,7.9,8.8,1
17,1,204.0,4641.0,100282.0,Our Lady's Catholic High School,N16 5AF,0.0,VA,Roman Catholic,COMP,...,8.3,7.9,8,7.4,6.3,7.2,8.1,7.3,7.5,3
33,1,206.0,4307.0,100455.0,Highbury Fields School,N5 1AR,0.0,CY,Does not apply,COMP,...,8.6,7.9,8.3,7.8,6.2,7.7,8.5,7.6,8.1,1
34,1,206.0,4324.0,100457.0,Elizabeth Garrett Anderson School,N1 9QG,0.0,CY,Does not apply,COMP,...,8.3,7.4,8,7.5,6.1,7.3,8.2,7.3,7.9,2
40,1,207.0,4801.0,100503.0,All Saints Catholic College,W10 6EL,0.0,VA,Roman Catholic,COMP,...,9,8.1,8.3,8,6.4,7.9,8.8,7.9,8.1,3
46,1,208.0,5400.0,100637.0,La Retraite Roman Catholic Girls' School,SW12 0AB,0.0,VA,Roman Catholic,COMP,...,9.5,8.6,8.8,8.7,7.8,8.4,9.5,8.6,8.8,2
48,1,208.0,5404.0,137966.0,St Martin in the Fields High School for Girls,SW2 3UP,0.0,ACC,Church of England,COMP,...,8.9,8.2,8.4,7.4,6.8,6.9,8.7,7.2,7.6,3


In [40]:
df_data_issues.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 365 to 0
Data columns (total 11 columns):
column_name           278 non-null object
# nulls               278 non-null int64
# "NE"                278 non-null int64
# "SUPP"              278 non-null int64
# "NEW"               278 non-null int64
# "NA"                278 non-null int64
# "LOWCOV"            278 non-null int64
# "NP"                278 non-null int64
total_freq            278 non-null int64
pct_all_rows          278 non-null int64
column_description    278 non-null object
dtypes: int64(9), object(2)
memory usage: 26.1+ KB


In [41]:
df_data_issues.head()

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows,column_description
365,LANVAMEA_LO_PTQ_EE,0,752,1830,0,0,5,0,2587,82,English Baccalaureate Languages Value Added me...
256,P8MEA_EAL,0,150,1168,0,0,0,15,1333,42,Adjusted Progress 8 measure - pupils for whom ...
255,TP8ADJ_EAL,0,165,1168,0,0,0,0,1333,42,Number of pupils for whom English is an additi...
259,P8MEA_EAL_ORIG,0,165,1168,0,0,0,0,1333,42,Unadjusted Progress 8 measure - pupils for who...
254,P8PUP_EAL,0,149,1168,0,0,0,15,1332,42,Number of pupils for whom English is an additi...


In [42]:
df6[(df6['P8MEA_BOYS_ORIG'] == 'NE') & (df6['EGENDER'] == 'MIXED')]

,RECTYPE,LEA,ESTAB,URN,SCHNAME,PCODE,ICLOSE,NFTYPE,RELDENOM,ADMPOL,...,TAVENT_E_3NG_HI_PTQ_EE,TAVENT_E_3NG_FSM6CLA1A_PTQ_EE,TAVENT_E_3NG_NFSM6CLA1A_PTQ_EE,TAVENT_G_PTQ_EE,TAVENT_GLO_PTQ_EE,TAVENT_GAV_PTQ_EE,TAVENT_GHI_PTQ_EE,TAVENT_GFSM6CLA1A_PTQ_EE,TAVENT_GNFSM6CLA1A_PTQ_EE,P8_BANDING
9,1,203.0,4130.0,100183.0,Plumstead Manor School,SE18 1QF,0.0,CY,Does not apply,COMP,...,9.3,7.9,8.5,7.9,5.3,7.8,9.1,7.6,8.2,3
30,1,206.0,4001.0,143659.0,"City of London Academy, Highgate Hill",N19 3EU,0.0,F,None,COMP,...,8.3,8.3,7.9,7.9,7.3,8,8,8,7.6,3
355,1,318.0,4021.0,138461.0,Waldegrave School,TW2 5LH,0.0,ACC,Does not apply,COMP,...,9.3,7.5,9.1,8.9,6.8,8.6,9.3,7.5,9.1,1
1211,1,887.0,5445.0,136313.0,The Rochester Grammar School,ME1 3BY,0.0,ACC,None,SEL,...,10.5,9.9,10.3,10.3,NE,9.1,10.5,9.9,10.3,1
1508,1,916.0,4012.0,115720.0,Barnwood Park Arts College,GL4 3QU,0.0,FD,None,COMP,...,8.4,7.3,7.9,7.7,6.1,7.7,8.4,7.3,7.9,2
1908,1,837.0,5404.0,138193.0,Avonbourne College,BH7 6NY,0.0,ACC,None,MOD,...,11,9.2,9.8,9.5,7.2,9.4,10.8,9.1,9.7,2
2018,1,851.0,4002.0,139714.0,The Portsmouth Academy,PO1 5PF,0.0,AC,Does not apply,COMP,...,9.7,8,8.4,8,6.9,7.7,9.5,7.7,8.4,3
2464,1,881.0,5461.0,138834.0,Brentwood Ursuline Convent High School,CM14 4EX,0.0,ACC,Roman Catholic,COMP,...,9.9,9,9.4,9.4,7.6,9,9.9,9,9.4,1
3808,1,341.0,5403.0,104721.0,St Hilda's Church of England High School,L17 3AL,0.0,VA,Church of England,COMP,...,8,7,8,7.4,5,7.1,7.8,6.8,7.8,3


In [43]:
df6[df6['P8MEA_BOYS_ORIG'] == 'NE'][['URN', 'PBPUP','TP8ADJ_BOYS', 'ATT8SCROPENG_BOYS']][:10]

,URN,PBPUP,TP8ADJ_BOYS,ATT8SCROPENG_BOYS
1,100050.0,0%,NE,NE
7,100059.0,0%,NE,NE
9,100183.0,0%,NE,NE
13,100193.0,0%,NE,NE
16,137442.0,0%,NE,NE
17,100282.0,0%,NE,NE
30,143659.0,0%,NE,NE
33,100455.0,0%,NE,NE
34,100457.0,0%,NE,NE
40,100503.0,0%,NE,NE


In [44]:
df_item_freq2 = item_freq_in_col(df6,['NE','SUPP', 'NEW', 'NA', 'LOWCOV', 'NP'])
df_item_freq2.head(100)

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
223,LANVAMEA_LO_PTQ_EE,0,752,1830,0,0,5,0,2587,82
160,P8MEA_EAL,0,150,1168,0,0,0,15,1333,42
159,TP8ADJ_EAL,0,165,1168,0,0,0,0,1333,42
161,P8MEA_EAL_ORIG,0,165,1168,0,0,0,0,1333,42
158,P8PUP_EAL,0,149,1168,0,0,0,15,1332,42
154,ATT8SCREBAC_EAL,0,164,910,0,0,0,0,1074,34
157,ATT8SCROPENNG_EAL,0,164,910,0,0,0,0,1074,34
156,ATT8SCROPENG_EAL,0,164,910,0,0,0,0,1074,34
155,ATT8SCROPEN_EAL,0,164,910,0,0,0,0,1074,34
240,PTEBACCEAL_94,0,149,910,0,0,0,15,1074,34


In [45]:
# for girls

girls_NE_cols = [
'NUMGIRLS',
'EBACCAPS_GIRLS',
'ATT8SCR_GIRLS',
'ATT8SCRENG_GIRLS',
'ATT8SCRMAT_GIRLS',
'ATT8SCREBAC_GIRLS',
'ATT8SCROPEN_GIRLS',
'ATT8SCROPENG_GIRLS',
'ATT8SCROPENNG_GIRLS',
'P8PUP_GIRLS',
'TP8ADJ_GIRLS',
'P8MEA_GIRLS',
'P8MEA_GIRLS_ORIG',
'PGEBACC_E_PTQ_EE',
'PGEBACC_94',
'PGEBACC_95',
'PGL2BASICS_94',
'PGL2BASICS_95']

In [46]:
# for boys...

boys_NE_cols = [
'NUMBOYS',
'PBPUP',
'ATT8SCR_BOYS',
'ATT8SCRENG_BOYS',
'ATT8SCRMAT_BOYS',
'ATT8SCREBAC_BOYS',
'ATT8SCROPEN_BOYS',
'ATT8SCROPENG_BOYS',
'ATT8SCROPENNG_BOYS',
'P8PUP_BOYS',
'TP8ADJ_BOYS',
'P8MEA_BOYS',
'P8MEA_BOYS_ORIG',
'PBEBACC_E_PTQ_EE',
'PBEBACC_94',
'PBEBACC_95',
'PBL2BASICS_94',
'PBL2BASICS_95']

In [47]:
boys_NE_cols

['NUMBOYS',
 'PBPUP',
 'ATT8SCR_BOYS',
 'ATT8SCRENG_BOYS',
 'ATT8SCRMAT_BOYS',
 'ATT8SCREBAC_BOYS',
 'ATT8SCROPEN_BOYS',
 'ATT8SCROPENG_BOYS',
 'ATT8SCROPENNG_BOYS',
 'P8PUP_BOYS',
 'TP8ADJ_BOYS',
 'P8MEA_BOYS',
 'P8MEA_BOYS_ORIG',
 'PBEBACC_E_PTQ_EE',
 'PBEBACC_94',
 'PBEBACC_95',
 'PBL2BASICS_94',
 'PBL2BASICS_95']

In [48]:
# define a function to replace vals in columns in a dataframe with specific value

def replace_vals(df, list_cols, word, val):
    '''Replaces values of 'word' in a list of columns (list_cols) in a df with val
    Notes:
    1) Will return just the original value for non-word values in the column
    2) List_cols must be a list
    '''
    for col in list_cols:
        df[col] = df[col].apply(lambda x: val if x == word else x)


In [49]:
# run the above function

replace_vals(df6, boys_NE_cols,'NE',0)
replace_vals(df6, girls_NE_cols, 'NE', 0)

In [50]:
df6[df6['P8PUP_BOYS'] == 'SUPP'].loc[:,['URN', 'EGENDER', 'PBPUP']]

,URN,EGENDER,PBPUP
52,100741.0,GIRLS,1%
184,102782.0,GIRLS,0%
365,103013.0,GIRLS,1%
1172,140941.0,MIXED,86%
1206,141012.0,MIXED,80%
1215,137498.0,MIXED,50%
1239,142863.0,MIXED,87%
1468,130555.0,MIXED,75%
2309,137259.0,GIRLS,1%
2398,141749.0,MIXED,92%


In [51]:
df_item_freq2 = item_freq_in_col(df6,['NE','SUPP', 'NEW', 'NA', 'LOWCOV', 'NP'])
df_item_freq2[:100]

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
223,LANVAMEA_LO_PTQ_EE,0,752,1830,0,0,5,0,2587,82
159,TP8ADJ_EAL,0,165,1168,0,0,0,0,1333,42
160,P8MEA_EAL,0,150,1168,0,0,0,15,1333,42
161,P8MEA_EAL_ORIG,0,165,1168,0,0,0,0,1333,42
158,P8PUP_EAL,0,149,1168,0,0,0,15,1332,42
151,ATT8SCR_EAL,0,149,910,0,0,0,15,1074,34
157,ATT8SCROPENNG_EAL,0,164,910,0,0,0,0,1074,34
156,ATT8SCROPENG_EAL,0,164,910,0,0,0,0,1074,34
155,ATT8SCROPEN_EAL,0,164,910,0,0,0,0,1074,34
154,ATT8SCREBAC_EAL,0,164,910,0,0,0,0,1074,34


### <font color='green'>iii) Admissions Policy: change blanks to 'OTHER NON SEL'</font>

In [52]:
# replace 'ADMPOL' NANs with 'OTHER NON SEL' for 2017 and 'COMP' for pre-2017

df6['ADMPOL_2017'] = df6['ADMPOL_2017'].fillna('OTHER NON SEL')
df6['ADMPOL'] = df6['ADMPOL'].fillna('COMP')

### <font color='green'>iv) P8 Banding: allocate each 'SUPP' school as either 1, 3 or 5 Banding, based on government scoring</font>

In [53]:
# write a function for working out which Progress 8 (P8) Banding a school should be:

def sort_p8_band(x):
    if x < -0.5:
        try:
            return 5
        except:
            return np.nan
    elif x <0.5:
        try:
            return 3
        except:
            return np.nan
    elif x >= 0.5:
        try:
            return 1
        except:
            return np.nan
    else:
        return np.nan

In [54]:
# apply the function above to the main DataFrame

df6['P8_BANDING'] = df6[['P8MEA', 'P8_BANDING']].apply(lambda x: sort_p8_band(x[0]) if x[1]=='SUPP' else x[1], axis=1)
df6.head(3)

,RECTYPE,LEA,ESTAB,URN,SCHNAME,PCODE,ICLOSE,NFTYPE,RELDENOM,ADMPOL,...,TAVENT_E_3NG_HI_PTQ_EE,TAVENT_E_3NG_FSM6CLA1A_PTQ_EE,TAVENT_E_3NG_NFSM6CLA1A_PTQ_EE,TAVENT_G_PTQ_EE,TAVENT_GLO_PTQ_EE,TAVENT_GAV_PTQ_EE,TAVENT_GHI_PTQ_EE,TAVENT_GFSM6CLA1A_PTQ_EE,TAVENT_GNFSM6CLA1A_PTQ_EE,P8_BANDING
0,1,202.0,4104.0,100049.0,Haverstock School,NW3 2BQ,0.0,CY,Does not apply,COMP,...,7.7,7.2,7.4,7.1,6.3,6.9,7.7,7,7.3,3
1,1,202.0,4166.0,100050.0,Parliament Hill School,NW5 1RL,0.0,CY,Does not apply,COMP,...,8.8,8.6,8.4,8.4,6.9,8.1,8.7,8.5,8.3,1
3,1,202.0,4196.0,100051.0,Regent High School,NW1 1RX,0.0,CY,Does not apply,COMP,...,8.3,7.8,7.9,7.2,6.4,7,8,7.2,7.2,4


In [55]:
# check that function has worked correctly
df6['P8_BANDING'].astype(int).value_counts()

3    1196
4     594
2     512
5     445
1     418
Name: P8_BANDING, dtype: int64

In [56]:
# check that function has worked correctly

df6[['P8MEA', 'P8_BANDING']].head(3)

,P8MEA,P8_BANDING
0,-0.10,3
1,0.65,1
3,-0.26,4


In [57]:
df7 = df6.copy()

### <font color='green'>v) Any column that has any non-numerics remaining in it is removed</font>

In [58]:
df_item_freq3 = item_freq_in_col(df7,['NE','SUPP', 'NEW', 'NA', 'LOWCOV', 'NP'])
df_item_freq3.head(100)

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
223,LANVAMEA_LO_PTQ_EE,0,752,1830,0,0,5,0,2587,82
161,P8MEA_EAL_ORIG,0,165,1168,0,0,0,0,1333,42
160,P8MEA_EAL,0,150,1168,0,0,0,15,1333,42
159,TP8ADJ_EAL,0,165,1168,0,0,0,0,1333,42
158,P8PUP_EAL,0,149,1168,0,0,0,15,1332,42
61,EBACCAPS_EAL,0,149,910,0,0,0,15,1074,34
155,ATT8SCROPEN_EAL,0,164,910,0,0,0,0,1074,34
252,PTL2BASICSEAL_94,0,149,910,0,0,0,15,1074,34
261,PTL2BASICSEAL_95,0,149,910,0,0,0,15,1074,34
151,ATT8SCR_EAL,0,149,910,0,0,0,15,1074,34


In [59]:
#Pct_GCSEpup_Eng_addnl_lang

df_item_freq3.loc[df_item_freq3['column_name'] == 'PTEALGRP2']


,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
26,PTEALGRP2,0,0,0,0,0,0,15,15,0


In [60]:
# columns that will remain
df_item_freq3[df_item_freq3['total_freq'] == 0]

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
265,TAVENT_E_3NG_PTQ_EE,0,0,0,0,0,0,0,0,0
264,PTtripleSci_E,0,0,0,0,0,0,0,0,0
271,TAVENT_G_PTQ_EE,0,0,0,0,0,0,0,0,0
242,PT5EM_94,0,0,0,0,0,0,0,0,0
243,PT5EM_94_18,0,0,0,0,0,0,0,0,0
244,PTANYQ_PTQ_EE,0,0,0,0,0,0,0,0,0
263,PTmultiLan_E,0,0,0,0,0,0,0,0,0
0,RECTYPE,0,0,0,0,0,0,0,0,0
1,LEA,0,0,0,0,0,0,0,0,0
20,KS2APS,0,0,0,0,0,0,0,0,0


In [61]:
# create list of all columns to be dropped (because they contain non-numeric values)

drop_cols = []
for row in range(0, len(df_item_freq3[df_item_freq3['total_freq'] >0])):
    drop_cols.append(df_item_freq3.values[row][0])

print(drop_cols)
print(len(drop_cols))

['LANVAMEA_LO_PTQ_EE', 'P8MEA_EAL_ORIG', 'P8MEA_EAL', 'TP8ADJ_EAL', 'P8PUP_EAL', 'EBACCAPS_EAL', 'ATT8SCROPEN_EAL', 'PTL2BASICSEAL_94', 'PTL2BASICSEAL_95', 'ATT8SCR_EAL', 'ATT8SCRENG_EAL', 'ATT8SCRMAT_EAL', 'ATT8SCREBAC_EAL', 'PTEBACCEAL_95', 'ATT8SCROPENG_EAL', 'ATT8SCROPENNG_EAL', 'PTEBACCEAL_94', 'PTEBACC_EEAL_PTQ_EE', 'LANVAMEA_FSM6CLA1A_PTQ_EE', 'HUMVAMEA_LO_PTQ_EE', 'LANVAMEA_AV_PTQ_EE', 'SCIVAMEA_LO_PTQ_EE', 'PTBASICSLO_95', 'PTBASICSLO_94', 'EBACCAPS_LO', 'P8MEA_LO', 'TP8ADJ_LO', 'TEBACC_ELO_PTQ_EE', 'PTEBACCLO_95', 'PTEBACCLO_94', 'TAVENT_GLO_PTQ_EE', 'ATT8SCR_LO', 'P8PUP_LO', 'TAVENT_E_3NG_LO_PTQ_EE', 'PTEBACC_ELO_PTQ_EE', 'P8MEA_LO_ORIG', 'LANVAMEA_HI_PTQ_EE', 'LANVAMEA_NFSM6CLA1A_PTQ_EE', 'HUMVAMEA_FSM6CLA1A_PTQ_EE', 'LANVAMEA_PTQ_EE', 'HUMVAMEA_AV_PTQ_EE', 'SCIVAMEA_FSM6CLA1A_PTQ_EE', 'HUMVAMEA_NFSM6CLA1A_PTQ_EE', 'TEBACLAN_95', 'LANVACOV_PTQ_EE', 'PTEBACLAN_95', 'PTEBACLAN91', 'PTEBACLAN_94', 'TEBACLAN_94', 'SCIVAMEA_AV_PTQ_EE', 'DIFFN_P8MEA', 'P8MEAOPEN_FSM6CLA1A', 'P8ME

In [62]:
len(df7.columns)

278

In [63]:
# drop the above columns

df8 = df7.drop(drop_cols, axis=1)
len(df8.columns)

74

In [64]:
df8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3165 entries, 0 to 5580
Data columns (total 74 columns):
RECTYPE                3165 non-null int64
LEA                    3165 non-null float64
ESTAB                  3165 non-null float64
URN                    3165 non-null float64
SCHNAME                3165 non-null object
PCODE                  3165 non-null object
ICLOSE                 3165 non-null float64
NFTYPE                 3165 non-null object
RELDENOM               3165 non-null object
ADMPOL                 3165 non-null object
ADMPOL_2017            3165 non-null object
EGENDER                3165 non-null object
FEEDER                 3165 non-null float64
TABKS2                 3165 non-null float64
TAB1618                3165 non-null float64
TPUP                   3165 non-null float64
PBPUP                  3165 non-null object
KS2APS                 3165 non-null object
PTPRIORLO              3165 non-null object
PTPRIORAV              3165 non-null object
PTPRIO

In [65]:
# check that all columns now have no non-numeric fields

df_item_freq4 = item_freq_in_col(df8,['NE','SUPP', 'NEW', 'NA', 'LOWCOV', 'NP'])
df_item_freq4.head()

,column_name,# nulls,"# ""NE""","# ""SUPP""","# ""NEW""","# ""NA""","# ""LOWCOV""","# ""NP""",total_freq,pct_all_rows
0,RECTYPE,0,0,0,0,0,0,0,0,0
55,PTEBACENG_94,0,0,0,0,0,0,0,0,0
53,PTEBACHUM_E_PTQ_EE,0,0,0,0,0,0,0,0,0
52,PTEBAC2SCI_E_PTQ_EE,0,0,0,0,0,0,0,0,0
51,PTEBACMAT_E_PTQ_EE,0,0,0,0,0,0,0,0,0


<a id='8'></a>

# <font color='blue'>8) Convert all relevant columns to floats</blue>

### <font color='green'>a) Display the dtypes of all the rows</blue>

In [66]:
pd.set_option('display.max_rows', 278)
df8.dtypes

RECTYPE                  int64
LEA                    float64
ESTAB                  float64
URN                    float64
SCHNAME                 object
PCODE                   object
ICLOSE                 float64
NFTYPE                  object
RELDENOM                object
ADMPOL                  object
ADMPOL_2017             object
EGENDER                 object
FEEDER                 float64
TABKS2                 float64
TAB1618                float64
TPUP                   float64
PBPUP                   object
KS2APS                  object
PTPRIORLO               object
PTPRIORAV               object
PTPRIORHI               object
PTFSM6CLA1A             object
PTNOTFSM6CLA1A          object
ATT8SCR                 object
ATT8SCRENG              object
ATT8SCRMAT              object
ATT8SCREBAC             object
ATT8SCROPEN             object
ATT8SCROPENG            object
ATT8SCROPENNG           object
AVGEBACFILL             object
AVGOPENFILL             object
P8PUP   

In [67]:
df8.get_dtype_counts()

/Users/jacktyler-whittle/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: `get_dtype_counts` has been deprecated and will be removed in a future version. For DataFrames use `.dtypes.value_counts()
  """Entry point for launching an IPython kernel.


int64       1
float64     9
object     64
dtype: int64

### <font color='green'>b) Convert columns to floats</blue>

Output of this section: dataset with columns all formatted appropriately

In [68]:
list(df8.columns)

['RECTYPE',
 'LEA',
 'ESTAB',
 'URN',
 'SCHNAME',
 'PCODE',
 'ICLOSE',
 'NFTYPE',
 'RELDENOM',
 'ADMPOL',
 'ADMPOL_2017',
 'EGENDER',
 'FEEDER',
 'TABKS2',
 'TAB1618',
 'TPUP',
 'PBPUP',
 'KS2APS',
 'PTPRIORLO',
 'PTPRIORAV',
 'PTPRIORHI',
 'PTFSM6CLA1A',
 'PTNOTFSM6CLA1A',
 'ATT8SCR',
 'ATT8SCRENG',
 'ATT8SCRMAT',
 'ATT8SCREBAC',
 'ATT8SCROPEN',
 'ATT8SCROPENG',
 'ATT8SCROPENNG',
 'AVGEBACFILL',
 'AVGOPENFILL',
 'P8PUP',
 'TP8ADJ',
 'P8MEACOV',
 'P8MEA',
 'P8CILOW',
 'P8CIUPP',
 'P8MEA_ORIG',
 'P8MEAENG',
 'P8MEAMAT',
 'P8MEAEBAC',
 'P8MEAOPEN',
 'PTL2BASICS_94',
 'PTL2BASICS_95',
 'TOTEBACCAPS',
 'EBACCAPS',
 'PTEBACC_E_PTQ_EE',
 'PTEBACC_94',
 'PTEBACC_95',
 'PTEBACENG_E_PTQ_EE',
 'PTEBACMAT_E_PTQ_EE',
 'PTEBAC2SCI_E_PTQ_EE',
 'PTEBACHUM_E_PTQ_EE',
 'PTEBACLAN_E_PTQ_EE',
 'PTEBACENG_94',
 'PTEBACENG_95',
 'PTEBACMAT_94',
 'PTEBACMAT_95',
 'TEBACENG_94',
 'TEBACENG_95',
 'TEBACMAT_94',
 'TEBACMAT_95',
 'PTEBACC91',
 'PTEBACENG91',
 'PTEBACMAT91',
 'PT5EM_94',
 'PT5EM_94_18',
 'PTANYQ

In [69]:
# Split out DataFrame into two files: one with columns to convert to floats (df9_to_convert), and the
# other consisting of data already formatted correctly (df9_text)

df9_to_convert = df8.loc[:,'FEEDER':]
df9_text = df8.loc[:,:'EGENDER']
df9_to_convert['URN'] = df7['URN']
df9_to_convert.head()

,FEEDER,TABKS2,TAB1618,TPUP,PBPUP,KS2APS,PTPRIORLO,PTPRIORAV,PTPRIORHI,PTFSM6CLA1A,...,PTEBACMAT91,PT5EM_94,PT5EM_94_18,PTANYQ_PTQ_EE,PTmultiLan_E,PTtripleSci_E,TAVENT_E_3NG_PTQ_EE,TAVENT_G_PTQ_EE,P8_BANDING,URN
0,0.0,0.0,1.0,197.0,58%,28.1,12%,56%,32%,61%,...,98%,57%,57%,99%,5%,17%,7.3,7.1,3,100049.0
1,1.0,0.0,1.0,168.0,0%,29.5,8%,40%,52%,45%,...,99%,77%,77%,100%,8%,33%,8.5,8.4,1,100050.0
3,0.0,0.0,1.0,133.0,50%,27.4,19%,51%,30%,68%,...,94%,42%,42%,99%,2%,20%,7.8,7.2,4,100051.0
4,0.0,0.0,1.0,184.0,56%,28,19%,44%,37%,49%,...,96%,51%,51%,98%,9%,22%,9.2,8.4,3,100052.0
5,1.0,0.0,1.0,161.0,70%,28.9,11%,46%,43%,52%,...,96%,59%,59%,99%,4%,24%,7.8,7.7,3,100053.0


In [70]:
# write function to change number formatted as a string and as 'xx%' into a float

def change_string(x): 
    if type(x) == str: 
        try:
            return float(x.replace("%",""))
        except:
            return x
    else:
        return x
    

In [71]:
# convert columns to numbers by removing % sign and converting to float

for col in df9_to_convert.columns:
    df9_to_convert[col] = df9_to_convert[col].apply(change_string)

In [72]:
df9_to_convert.head(3)

,FEEDER,TABKS2,TAB1618,TPUP,PBPUP,KS2APS,PTPRIORLO,PTPRIORAV,PTPRIORHI,PTFSM6CLA1A,...,PTEBACMAT91,PT5EM_94,PT5EM_94_18,PTANYQ_PTQ_EE,PTmultiLan_E,PTtripleSci_E,TAVENT_E_3NG_PTQ_EE,TAVENT_G_PTQ_EE,P8_BANDING,URN
0,0.0,0.0,1.0,197.0,58.0,28.1,12.0,56.0,32.0,61.0,...,98.0,57.0,57.0,99.0,5.0,17.0,7.3,7.1,3.0,100049.0
1,1.0,0.0,1.0,168.0,0.0,29.5,8.0,40.0,52.0,45.0,...,99.0,77.0,77.0,100.0,8.0,33.0,8.5,8.4,1.0,100050.0
3,0.0,0.0,1.0,133.0,50.0,27.4,19.0,51.0,30.0,68.0,...,94.0,42.0,42.0,99.0,2.0,20.0,7.8,7.2,4.0,100051.0


In [73]:
# check data types of columns

df9_to_convert.get_dtype_counts()

/Users/jacktyler-whittle/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: `get_dtype_counts` has been deprecated and will be removed in a future version. For DataFrames use `.dtypes.value_counts()
  This is separate from the ipykernel package so we can avoid doing imports until


float64    63
dtype: int64

### <font color='green'>c) Finalise Datasets</font>

In [74]:
df_final_base_feat = pd.merge(df9_text, df9_to_convert, on='URN', how='left', left_index=True)
df_final_base_feat.head(10)

,RECTYPE,LEA,ESTAB,URN,SCHNAME,PCODE,ICLOSE,NFTYPE,RELDENOM,ADMPOL,...,PTEBACENG91,PTEBACMAT91,PT5EM_94,PT5EM_94_18,PTANYQ_PTQ_EE,PTmultiLan_E,PTtripleSci_E,TAVENT_E_3NG_PTQ_EE,TAVENT_G_PTQ_EE,P8_BANDING
0,1,202.0,4104.0,100049.0,Haverstock School,NW3 2BQ,0.0,CY,Does not apply,COMP,...,98.0,98.0,57.0,57.0,99.0,5.0,17.0,7.3,7.1,3.0
1,1,202.0,4166.0,100050.0,Parliament Hill School,NW5 1RL,0.0,CY,Does not apply,COMP,...,98.0,99.0,77.0,77.0,100.0,8.0,33.0,8.5,8.4,1.0
3,1,202.0,4196.0,100051.0,Regent High School,NW1 1RX,0.0,CY,Does not apply,COMP,...,95.0,94.0,42.0,42.0,99.0,2.0,20.0,7.8,7.2,4.0
4,1,202.0,4275.0,100052.0,Hampstead School,NW2 3RT,0.0,CY,Does not apply,COMP,...,97.0,96.0,51.0,51.0,98.0,9.0,22.0,9.2,8.4,3.0
5,1,202.0,4285.0,100053.0,Acland Burghley School,NW5 1UJ,0.0,CY,Does not apply,COMP,...,99.0,96.0,59.0,59.0,99.0,4.0,24.0,7.8,7.7,3.0
6,1,202.0,4652.0,100055.0,Maria Fidelis Catholic School FCJ,NW1 1LY,0.0,VA,Roman Catholic,COMP,...,99.0,98.0,45.0,45.0,100.0,8.0,25.0,7.9,7.8,4.0
7,1,202.0,5401.0,100059.0,La Sainte Union Catholic Secondary School,NW5 1RP,0.0,VA,Roman Catholic,COMP,...,100.0,99.0,80.0,80.0,100.0,8.0,28.0,8.4,8.4,1.0
8,1,203.0,4001.0,138245.0,Ark Greenwich Free School,SE18 4LH,0.0,F,None,COMP,...,100.0,99.0,60.0,60.0,100.0,2.0,66.0,8.6,8.6,2.0
9,1,203.0,4130.0,100183.0,Plumstead Manor School,SE18 1QF,0.0,CY,Does not apply,COMP,...,95.0,86.0,45.0,45.0,96.0,6.0,16.0,8.2,7.9,3.0
10,1,203.0,4243.0,137473.0,Corelli College,SE3 8EP,1.0,ACC,Does not apply,COMP,...,93.0,87.0,30.0,30.0,97.0,9.0,16.0,7.4,6.7,5.0


In [75]:
df_final_base_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3165 entries, 0 to 5580
Data columns (total 74 columns):
RECTYPE                3165 non-null int64
LEA                    3165 non-null float64
ESTAB                  3165 non-null float64
URN                    3165 non-null float64
SCHNAME                3165 non-null object
PCODE                  3165 non-null object
ICLOSE                 3165 non-null float64
NFTYPE                 3165 non-null object
RELDENOM               3165 non-null object
ADMPOL                 3165 non-null object
ADMPOL_2017            3165 non-null object
EGENDER                3165 non-null object
FEEDER                 3165 non-null float64
TABKS2                 3165 non-null float64
TAB1618                3165 non-null float64
TPUP                   3165 non-null float64
PBPUP                  3165 non-null float64
KS2APS                 3165 non-null float64
PTPRIORLO              3165 non-null float64
PTPRIORAV              3165 non-null float64
PT

In [76]:
df_final_base_feat.head()

,RECTYPE,LEA,ESTAB,URN,SCHNAME,PCODE,ICLOSE,NFTYPE,RELDENOM,ADMPOL,...,PTEBACENG91,PTEBACMAT91,PT5EM_94,PT5EM_94_18,PTANYQ_PTQ_EE,PTmultiLan_E,PTtripleSci_E,TAVENT_E_3NG_PTQ_EE,TAVENT_G_PTQ_EE,P8_BANDING
0,1,202.0,4104.0,100049.0,Haverstock School,NW3 2BQ,0.0,CY,Does not apply,COMP,...,98.0,98.0,57.0,57.0,99.0,5.0,17.0,7.3,7.1,3.0
1,1,202.0,4166.0,100050.0,Parliament Hill School,NW5 1RL,0.0,CY,Does not apply,COMP,...,98.0,99.0,77.0,77.0,100.0,8.0,33.0,8.5,8.4,1.0
3,1,202.0,4196.0,100051.0,Regent High School,NW1 1RX,0.0,CY,Does not apply,COMP,...,95.0,94.0,42.0,42.0,99.0,2.0,20.0,7.8,7.2,4.0
4,1,202.0,4275.0,100052.0,Hampstead School,NW2 3RT,0.0,CY,Does not apply,COMP,...,97.0,96.0,51.0,51.0,98.0,9.0,22.0,9.2,8.4,3.0
5,1,202.0,4285.0,100053.0,Acland Burghley School,NW5 1UJ,0.0,CY,Does not apply,COMP,...,99.0,96.0,59.0,59.0,99.0,4.0,24.0,7.8,7.7,3.0


<a id='9'></a>

# <font color='blue'>9) Export final (base) dataset</blue>

In [77]:
# EXPORT FILE TO CSV

df_final_base_feat.to_csv('data_files/final_base_feat_gcse_2018.csv')

END